# Gradient Boosting Model

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import cross_val_score, KFold
from xgboost.sklearn import XGBClassifier

from utils.metrics import ndcg_scorer

In [2]:
path = '../data/processed/'
prefix = 'processed_'
train_users = pd.read_csv(path + prefix + 'train_users.csv')

train_users.fillna(-1, inplace=True)
y_train = train_users['country_destination']
train_users.drop(['country_destination', 'id'], axis=1, inplace=True)

In [3]:
x_train = train_users.astype('int16').values
label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train)
del train_users

In [4]:
xgb = XGBClassifier(
    max_depth=8,
    learning_rate=0.16,
    n_estimators=55,
    gamma=0,
    min_child_weight=1,
    max_delta_step=0,
    subsample=1,
    colsample_bytree=1,
    colsample_bylevel=1,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    base_score=0.5,
    missing=None,
    silent=True,
    nthread=-1,
    seed=42
)
xgb.fit(x_train, encoded_y_train)

KeyboardInterrupt: 

In [ ]:
kf = KFold(len(x_train), n_folds=5, random_state=42)

score = cross_val_score(clf, x_train, encoded_y_train, cv=kf, scoring=ndcg_scorer)